In [1]:
import pandas as pd
import numpy as np
import nltk
import nltk.sentiment.util
from nltk.tokenize import sent_tokenize, word_tokenize
import matplotlib.pyplot as plt
import pprint
import re
import gensim
import gensim.corpora as corpora
import os
from gensim.utils import simple_preprocess
from nltk.corpus import stopwords
import string
import math

In [2]:
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

In [3]:
df = pd.read_csv('../../amazon_reviews_us_Grocery_v1_00_sentiment.csv').sample(50000)

df = df.drop(["marketplace", "customer_id", "product_parent", "product_category", "review_date"], axis=1)

df = df[~df['review_body'].isnull()]

df['review_body'] = df['review_headline'] + '. ' + df['review_body']

df["health_hazard"] = np.nan

df = df.set_index("review_id")

In [4]:
df

,product_id,product_title,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,polarity,subjectivity,health_hazard
review_id,,,,,,,,,,,,
R3KOE6LIHWSHJJ,B0065G59F4,"Ekobrew Cup, Refillable Cup for Keurig K-Cup B...",4,1,1,N,Y,Get the stainless version,Get the stainless version. The Ekobrew refill ...,0.468889,0.551111,NaN
R15E9LIPJO6XWZ,B00AFAO88K,Senseo Extra Dark Coffee Pods 96-count Pods - ...,5,0,0,N,Y,THE BEST COFFEE,THE BEST COFFEE. GREAT COFFEE SENSO IS STILL T...,0.552273,0.520076,NaN
R24G8JEGOCNTWF,B0074MURE6,"Tea Forte Loose Leaf Herbal Teas, 3.5 Ounce Te...",5,0,0,N,Y,Five Stars,Five Stars. excellent tea...love it,1.000000,1.000000,NaN
R1DPEW2JBPG0GS,B001ELL8BO,"Albanese Fresh Strawberry Gummi Bears, 5-Pound...",2,0,3,N,Y,"not really pink, more like a red","not really pink, more like a red. The gummies ...",-0.041667,0.326389,NaN
R2XBBNMXKPOVXF,B000B8EQVK,"Eden Foods Organic Cajun Rice and Beans, 15 oz",4,0,0,N,Y,Good easy meal,Good easy meal. Although this was slightly bla...,0.153704,0.409259,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
ROC5A2IGAIJUF,B00B9HTVIU,Starbucks House Blend Medium Roast 0.42 Oz (12...,4,0,0,N,Y,Very Tasty......,Very Tasty....... Husband liked this coffee......,0.600000,0.800000,NaN
RZZX6G4ATKR7V,B00JA9EH1K,"The Paleo Diet Bar ,Cranberry Almond, 2.47 Oun...",2,1,2,N,N,There isn't much I don't like to eat,There isn't much I don't like to eat. There is...,-0.238889,0.472222,NaN
R1VJPKZ5CS1P89,B00DSX5QLE,OFX53005 - Royal Dansk Cookies,5,0,0,N,Y,Five Stars,Five Stars. Love this cookies,0.500000,0.600000,NaN


In [5]:
neg_reviews = df.query("star_rating < 4", engine="python")

In [6]:
stop_words = stopwords.words('english')
stop_words.extend(['br'])
translator=str.maketrans('','',string.punctuation)

#neg_reviews["tokened_review"] = np.nan
all_tokens = []
#neg_reviews.astype({"tokened_review": 'object'}).dtypes
for index, row in neg_reviews.iterrows():
    text = row["review_body"].lower()
    text = text.translate(translator)
    text = word_tokenize(text)
    new_text = []
    for token in text:
        token = stemmer.stem(token)
        if token not in stop_words:
            new_text.append(token)
    all_tokens.append(new_text)
neg_reviews["tokened_review"] = all_tokens

<ipython-input-6-eba5c76ea0a6>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  neg_reviews["tokened_review"] = all_tokens


In [7]:
neg_reviews

,product_id,product_title,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,polarity,subjectivity,health_hazard,tokened_review
review_id,,,,,,,,,,,,,
R1DPEW2JBPG0GS,B001ELL8BO,"Albanese Fresh Strawberry Gummi Bears, 5-Pound...",2,0,3,N,Y,"not really pink, more like a red","not really pink, more like a red. The gummies ...",-0.041667,0.326389,NaN,"[realli, pink, like, red, gummi, red, pink, pi..."
R3HUB1B7AKIZ8H,B007Y5W4QM,"24 Teas Sampler Pack, Guaranteed 24 Different ...",3,0,0,N,Y,Sample of teas,Sample of teas. I considered switching from co...,0.115278,0.555556,NaN,"[sampl, tea, consid, switch, coffe, morn, tea,..."
R9GMPQNJVQX4W,B0007VWOHO,3 Pack of Emergency Water Boxes,1,2,4,N,N,Not nearly enough water,Not nearly enough water. The listing states th...,0.000000,0.500000,NaN,"[nearli, enough, water, list, state, thi, thre..."
R3T9UBST2ZFBKP,B002SI4USK,9GreenBox - Live 8 Braided Lucky Bamboo Plant ...,3,1,1,N,Y,Need the vase!,Need the vase!. Nice plant but didn't receive ...,0.600000,1.000000,NaN,"[need, vase, nice, plant, didnt, receiv, vase,..."
R3ML5RH5UI93JD,B00EDS8FZM,"Al Arz Tahini Sesame Seed Paste, 16 oz",3,0,2,N,Y,"Tastes very good, though there seems to be a b...","Tastes very good, though there seems to be a b...",0.127000,0.391000,NaN,"[tast, veri, good, though, seem, bit, tast, ve..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
R21FAYGWIMRX0P,B00H3KZMBM,Quinn Popcorn Microwave Pack,2,1,1,N,Y,Two Stars,Two Stars. I Love popcorn so very much but thi...,0.486667,0.486667,NaN,"[two, star, love, popcorn, veri, much, thi, on..."
R3A8XKRP4YMLS7,B001LXYA5Q,"mberry Miracle Fruit Tablets, 10-Count",1,1,2,N,Y,Waste of Time and Money,Waste of Time and Money. They barely worked! I...,0.028750,0.495000,NaN,"[wast, time, money, bare, work, got, differ, f..."
R9CVP2QBA8GKG,B002CPVTH4,Manitoba Harvest Hemp Pro 50 Protein Supplemen...,1,14,39,N,Y,Vile & Foul,Vile & Foul. I have never written a review bef...,0.050794,0.513988,NaN,"[vile, foul, never, written, review, befor, fe..."


In [8]:
hazardous_words=["rotten", "mouldy", "moldy", "mold", "mould", "sick", "dangerous", "diarrhea", "poisoning", "stale"]
for word in hazardous_words:
    neg_reviews.loc[[word in tokened_review for tokened_review in neg_reviews["tokened_review"]], "health_hazard"] = 1
    

C:\Users\stefa\AppData\Roaming\Python\Python38\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [9]:
num_labeled = len(neg_reviews.query("health_hazard == 1", engine="python"))
num_labeled

403

In [10]:
non_hazardous = neg_reviews.query("health_hazard != 1 and polarity < 0", engine="python").sample(num_labeled)
for i in list(non_hazardous.index):
    neg_reviews.loc[i, "health_hazard"] = 0

C:\Users\stefa\AppData\Roaming\Python\Python38\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [11]:
labeled = neg_reviews.query("health_hazard == 0 or health_hazard == 1", engine="python")
reviews = labeled.loc[:, 'tokened_review'].values
y = labeled.loc[:, 'health_hazard'].values

In [12]:
reviews

array([list(['realli', 'pink', 'like', 'red', 'gummi', 'red', 'pink', 'pictur', 'purchas', 'thi', 'candi', 'buffet', 'tabl', 'wed', 'theme', 'color', 'pink', 'white', 'havent', 'tri', 'candi', 'yet', 'im', 'sure', 'color', 'go', 'well', 'bride']),
       list(['fresh', 'addict', 'one', 'bought', 'amazon', 'though', 'look', 'ident', 'simpli', 'tast', 'good', 'one', 'ive', 'bought', 'costco', 'dont', 'seem', 'fresh', 'bought', 'august', 'expir', 'decemb', 'alreadi', 'tast', 'stale', 'august', 'contrast', 'befor', 'even', 'eat', 'half', 'bought', 'costco', 'octob', 'expir', 'date', 'april', 'much', 'fresher', 'im', 'go', 'troubl', 'finish', 'one', 'amazon', 'becaus', 'one', 'costco', 'much', 'better']),
       list(['one', 'star', 'flavor', 'soft', 'fast', 'deliveri', 'tasteless']),
       list(['dont', 'bother', 'worst', 'jordan', 'almond', 'ive', 'ever', 'tast', 'receiv', '5', 'pound', 'packag', 'last', 'week', 'threw', 'away', 'day', 'eat', 'hand', '34jordan', 'almond34', 'flavor', 'su

In [13]:
for i in range(len(reviews)):
    reviews[i] = ' '.join(reviews[i])

In [14]:
reviews

array(['realli pink like red gummi red pink pictur purchas thi candi buffet tabl wed theme color pink white havent tri candi yet im sure color go well bride',
       'fresh addict one bought amazon though look ident simpli tast good one ive bought costco dont seem fresh bought august expir decemb alreadi tast stale august contrast befor even eat half bought costco octob expir date april much fresher im go troubl finish one amazon becaus one costco much better',
       'one star flavor soft fast deliveri tasteless',
       'dont bother worst jordan almond ive ever tast receiv 5 pound packag last week threw away day eat hand 34jordan almond34 flavor sugar coat almond wast 2700',
       'good sunshin crawford cracker crisp dri veri tasti veri much like sunshin raisin cooki strongli prefer crawford garibaldi tast better like sunshin mine tast stale like review found',
       'doesnt tast quit like premad bottl stuff thi powder gatorad ok feel sick make store buy real thing work doesnt tast

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer
matrix = TfidfVectorizer(max_features=200)
X = matrix.fit_transform(reviews).toarray()
X

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.3080293 , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.10481997,
        0.14929222],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [16]:
# split train and test data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [17]:
from sklearn.naive_bayes import GaussianNB
classifier = GaussianNB()

#from sklearn.linear_model import LogisticRegression

#classifier = LogisticRegression()

classifier.fit(X_train, y_train)

# Predict Class
y_pred = classifier.predict(X_test)

# Accuracy 
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, y_pred)
print(accuracy)

0.8663366336633663


In [18]:
trial = neg_reviews.sample(100)

In [19]:
matrix.get_feature_names()

['absolut',
 'almond',
 'almost',
 'also',
 'amazon',
 'ani',
 'anyth',
 'arriv',
 'aw',
 'away',
 'back',
 'bad',
 'bag',
 'bar',
 'batch',
 'bean',
 'becaus',
 'befor',
 'better',
 'bit',
 'bitter',
 'bought',
 'box',
 'brand',
 'broken',
 'butter',
 'buy',
 'came',
 'candi',
 'cant',
 'case',
 'chees',
 'chip',
 'chocol',
 'coconut',
 'coffe',
 'color',
 'come',
 'compani',
 'contain',
 'cooki',
 'could',
 'couldnt',
 'cracker',
 'cup',
 'date',
 'day',
 'didnt',
 'differ',
 'disappoint',
 'doe',
 'doesnt',
 'dont',
 'dri',
 'drink',
 'eat',
 'effect',
 'end',
 'even',
 'ever',
 'everi',
 'expect',
 'expens',
 'experi',
 'expir',
 'feel',
 'find',
 'first',
 'flavor',
 'food',
 'found',
 'fresh',
 'get',
 'give',
 'go',
 'good',
 'got',
 'great',
 'green',
 'ha',
 'half',
 'hard',
 'honey',
 'hope',
 'horribl',
 'howev',
 'id',
 'im',
 'ingredi',
 'item',
 'ive',
 'kind',
 'know',
 'label',
 'last',
 'less',
 'let',
 'like',
 'littl',
 'local',
 'look',
 'lot',
 'love',
 'made',
 'm

In [20]:
trial = neg_reviews
trial_x = trial.loc[:,"tokened_review"].values
for i in range(len(trial_x)):
    trial_x[i] = ' '.join(trial_x[i])
trial_X = matrix.fit_transform(trial_x).toarray()
trial_y = classifier.predict(trial_X)
trial_y

array([1., 0., 1., ..., 1., 0., 0.])

In [21]:
neg_reviews["health_hazard"] = trial_y

<ipython-input-21-56d181f868ab>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  neg_reviews["health_hazard"] = trial_y


In [22]:
neg_reviews

,product_id,product_title,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,polarity,subjectivity,health_hazard,tokened_review
review_id,,,,,,,,,,,,,
R1DPEW2JBPG0GS,B001ELL8BO,"Albanese Fresh Strawberry Gummi Bears, 5-Pound...",2,0,3,N,Y,"not really pink, more like a red","not really pink, more like a red. The gummies ...",-0.041667,0.326389,1.0,realli pink like red gummi red pink pictur pur...
R3HUB1B7AKIZ8H,B007Y5W4QM,"24 Teas Sampler Pack, Guaranteed 24 Different ...",3,0,0,N,Y,Sample of teas,Sample of teas. I considered switching from co...,0.115278,0.555556,0.0,sampl tea consid switch coffe morn tea want tr...
R9GMPQNJVQX4W,B0007VWOHO,3 Pack of Emergency Water Boxes,1,2,4,N,N,Not nearly enough water,Not nearly enough water. The listing states th...,0.000000,0.500000,1.0,nearli enough water list state thi three day s...
R3T9UBST2ZFBKP,B002SI4USK,9GreenBox - Live 8 Braided Lucky Bamboo Plant ...,3,1,1,N,Y,Need the vase!,Need the vase!. Nice plant but didn't receive ...,0.600000,1.000000,0.0,need vase nice plant didnt receiv vase hold
R3ML5RH5UI93JD,B00EDS8FZM,"Al Arz Tahini Sesame Seed Paste, 16 oz",3,0,2,N,Y,"Tastes very good, though there seems to be a b...","Tastes very good, though there seems to be a b...",0.127000,0.391000,1.0,tast veri good though seem bit tast veri good ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
R21FAYGWIMRX0P,B00H3KZMBM,Quinn Popcorn Microwave Pack,2,1,1,N,Y,Two Stars,Two Stars. I Love popcorn so very much but thi...,0.486667,0.486667,0.0,two star love popcorn veri much thi one good
R3A8XKRP4YMLS7,B001LXYA5Q,"mberry Miracle Fruit Tablets, 10-Count",1,1,2,N,Y,Waste of Time and Money,Waste of Time and Money. They barely worked! I...,0.028750,0.495000,0.0,wast time money bare work got differ food said...
R9CVP2QBA8GKG,B002CPVTH4,Manitoba Harvest Hemp Pro 50 Protein Supplemen...,1,14,39,N,Y,Vile & Foul,Vile & Foul. I have never written a review bef...,0.050794,0.513988,1.0,vile foul never written review befor feel must...


In [23]:
dangerous = neg_reviews.query("health_hazard == 1", engine="python")
dangerous

,product_id,product_title,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,polarity,subjectivity,health_hazard,tokened_review
review_id,,,,,,,,,,,,,
R1DPEW2JBPG0GS,B001ELL8BO,"Albanese Fresh Strawberry Gummi Bears, 5-Pound...",2,0,3,N,Y,"not really pink, more like a red","not really pink, more like a red. The gummies ...",-0.041667,0.326389,1.0,realli pink like red gummi red pink pictur pur...
R9GMPQNJVQX4W,B0007VWOHO,3 Pack of Emergency Water Boxes,1,2,4,N,N,Not nearly enough water,Not nearly enough water. The listing states th...,0.000000,0.500000,1.0,nearli enough water list state thi three day s...
R3ML5RH5UI93JD,B00EDS8FZM,"Al Arz Tahini Sesame Seed Paste, 16 oz",3,0,2,N,Y,"Tastes very good, though there seems to be a b...","Tastes very good, though there seems to be a b...",0.127000,0.391000,1.0,tast veri good though seem bit tast veri good ...
R1SJF3QE4Y8TCV,B001EO5U42,"La Tourangelle Roasted Hazelnut Oil, 16.9 Ounc...",2,40,46,N,Y,"Tasty Hazelnut Oil- Not Anymore, & Made in Cal...","Tasty Hazelnut Oil- Not Anymore, & Made in Cal...",0.099792,0.485000,1.0,tasti hazelnut oil anymor made california thi ...
RTQNAN01BWPPU,B000EMOCDA,"Nature Valley Sweet & Salty Nut Granola Bars, ...",1,1,1,N,Y,Not fresh,Not fresh. I am addicted to these. But the on...,-0.016667,0.478381,1.0,fresh addict one bought amazon though look ide...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
R2AR81U215LM6H,B000V9PHEY,Camano Island Coffee Roasters Honduras Dark Ro...,1,1,1,N,N,kinda nutty,kinda nutty. I have tried several honduran org...,0.283333,0.487654,1.0,kinda nutti tri sever honduran organ coffe smo...
R19LHJ9E8SG0YM,B008LBCYXQ,Rose Petal Black Tea,1,0,0,N,Y,However the sample of the White Jasmine was re...,However the sample of the White Jasmine was re...,0.338889,0.491204,1.0,howev sampl white jasmin wa realli great wa bl...
R1ABPIBQUFXXZE,B0039ZIB5E,"SweetGourmet Hazer Baba Mixed Turkish Delight,...",1,0,6,N,Y,Not what I expected,Not what I expected. It was not what I expecte...,0.116667,0.466667,1.0,expect wa expect orang jelli candi sprinkl sug...


In [24]:
#LDA
data_filtered = dangerous['review_body'].values
data_filtered

array(["not really pink, more like a red. The gummies are more red than the pink as pictured. I purchased this for a candy buffet table at a wedding. The theme colors are pink and white. I haven't tried the candies yet, but I'm not sure if the color will go well with the bride.",
       'Not nearly enough water. The listing states this is a three day supply of emergency water for one person. These boxes are only 8.45 ounces each.',
       "Tastes very good, though there seems to be a bit of .... Tastes very good, though there seems to be a bit of hype about it. Much thinner than anything else out there. Not good for baking because it's too liquidy. Keep that in mind when you compare costs to other brands. Will not buy again.",
       ...,
       "Not what I expected. It was not what I expected at all.  It's just orange jellied candy sprinkled with sugar--the kind that was given out for halloween back in the 70s and 80s cause it was cheap.",
       "Don't buy!. I love cornbread, but I h

In [25]:
def sent_to_words(sentences):
    for sentence in sentences:
        sentence = sentence.lower()
        sentence = sentence.translate(translator)
        yield(nltk.word_tokenize(str(sentence)))

data_words = list(sent_to_words(data_filtered))
clean_data = []
for i in range(len(data_words)):
    clean_review = []
    for j in range(len(data_words[i])):
        if data_words[i][j] not in stop_words:
            clean_review.append(data_words[i][j])
    clean_data.append(clean_review)

In [26]:
# Create Dictionary
id2word = corpora.Dictionary(clean_data)
# Create Corpus
texts = clean_data
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]
# View
pprint.pprint(corpus[:5])

[[(0, 1),
  (1, 1),
  (2, 1),
  (3, 1),
  (4, 1),
  (5, 1),
  (6, 1),
  (7, 1),
  (8, 1),
  (9, 1),
  (10, 1),
  (11, 1),
  (12, 3),
  (13, 1),
  (14, 1),
  (15, 2),
  (16, 1),
  (17, 1),
  (18, 1),
  (19, 1),
  (20, 1),
  (21, 1),
  (22, 1),
  (23, 1)],
 [(24, 1),
  (25, 1),
  (26, 1),
  (27, 1),
  (28, 1),
  (29, 1),
  (30, 1),
  (31, 1),
  (32, 1),
  (33, 1),
  (34, 1),
  (35, 1),
  (36, 1),
  (37, 2)],
 [(38, 1),
  (39, 1),
  (40, 2),
  (41, 1),
  (42, 1),
  (43, 1),
  (44, 1),
  (45, 1),
  (46, 3),
  (47, 1),
  (48, 1),
  (49, 1),
  (50, 1),
  (51, 1),
  (52, 2),
  (53, 2),
  (54, 1),
  (55, 2)],
 [(9, 1),
  (30, 1),
  (31, 1),
  (46, 2),
  (56, 1),
  (57, 1),
  (58, 1),
  (59, 1),
  (60, 1),
  (61, 1),
  (62, 2),
  (63, 1),
  (64, 1),
  (65, 1),
  (66, 1),
  (67, 1),
  (68, 1),
  (69, 1),
  (70, 1),
  (71, 1),
  (72, 1),
  (73, 1),
  (74, 1),
  (75, 1),
  (76, 1),
  (77, 2),
  (78, 1),
  (79, 1),
  (80, 2),
  (81, 1),
  (82, 1),
  (83, 4),
  (84, 2),
  (85, 1),
  (86, 1),
  (87, 

In [28]:
# number of topics
num_topics = 3
# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=num_topics,
                                       minimum_probability=0.1,
                                       workers=3,
                                       passes=2)
# Print the Keyword in the 10 topics
pprint.pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[(0,
  '0.012*"like" + 0.009*"taste" + 0.007*"good" + 0.007*"one" + 0.006*"flavor" '
  '+ 0.005*"buy" + 0.005*"product" + 0.004*"didnt" + 0.004*"get" + '
  '0.004*"really"'),
 (1,
  '0.012*"like" + 0.010*"product" + 0.008*"taste" + 0.007*"would" + '
  '0.006*"good" + 0.006*"chocolate" + 0.006*"flavor" + 0.006*"much" + '
  '0.005*"one" + 0.005*"candy"'),
 (2,
  '0.013*"like" + 0.011*"good" + 0.009*"taste" + 0.008*"one" + 0.007*"flavor" '
  '+ 0.006*"better" + 0.006*"coffee" + 0.006*"would" + 0.005*"something" + '
  '0.005*"dont"')]


In [47]:
#from itertools import chain
#lda_corpus = lda_model[corpus]

# Find the threshold, let's set the threshold to be 1/#clusters,
# To prove that the threshold is sane, we average the sum of all probabilities:
#scores = list(chain(*[[score for topic_id,score in topic] for topic in [doc for doc in lda_corpus]]))
#threshold = sum(scores)/len(scores)


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [29]:
import pickle 
import pyLDAvis
import pyLDAvis.gensim_models
data_vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)

pyLDAvis.display(data_vis)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
